In [121]:
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras.callbacks
from keras.layers import Dense, Dropout, Activation, TimeDistributed, Input
from keras.layers.recurrent import LSTM
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
import pandas as pd
from itertools import groupby
import numpy.ma as ma
import catboost
from sklearn import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [5]:
event_types = ["bc613fb9d5", "bd18c260dd", "78a254eb1a", "60d7fad2cc", 
               "9f449c8a24", "44f0b93123", "c7863fbab6", "287a406e15", 
               "87d86c4ba1", "718ac49d0b", "1bea63552c", "4739c12685", 
               "09dff9a4e6", "e1218bb17f", "30fe294f41", "f3adcadc86", 
               "8fb049c69a", "a29c238412", "e5e18713a0", "bbfc7ae3f7", 
               "9c67e951dd", "5d5d31ecb1", "81b0435926", "8ccd550d04", 
               "416674c7cf", "96c40ef2e4", "221f9b90a3", "4234879f4b", 
               "444d9e80a6", "8b6000cce4", "d8c799feca"]

In [24]:
def process_row(train_part):
    train_part = json.loads(train_part)
    x_row = []
    y_row = []
    for data_point in train_part:
        x_data_point = [
            np.mean(data_point["lengths_history"]),
            data_point['user_state']["age"],
            data_point['user_state']['height'],
            data_point['user_state']['weight'],
            data_point['user_state']['period_estimate'],
            data_point['user_state']['luteal_estimate'],
            data_point['user_state']['cycle_estimate'],
            int(data_point['period_passed'])
        ]
        evts_map = {x['type'] : np.mean(x['value']) for x in data_point['events']}
        for evt in event_types:
            if evt in evts_map:
                x_data_point.append(np.mean(evts_map[evt]))
            else:
                x_data_point.append(0)
        x_data_point = [float('nan') if x is None else x for x in x_data_point]
        x_row.append(np.array(x_data_point))
        if 'label' in data_point:
            lbl = data_point['label']
            y_row.append(lbl)
    return np.array(x_row), np.array(y_row)

In [151]:
train_parts = []
y_train_parts = []
test_parts = []
i = 0
with open('/Users/vita/Downloads/flo/train_subsample.jsonl', 'r') as train_file, \
        open('/Users/vita/Downloads/flo/test_subsample.jsonl', 'r') as test_file:
    for train_part, test_part in zip(tqdm(train_file), test_file):
        i = i + 1
        if i > 10000:
            break
            
        train_x_row, train_y_row = process_row(train_part)
        train_parts.append(train_x_row)
        y_train_parts.append(train_y_row)

        test_x_row, test_y_row = process_row(test_part)
        test_parts.append(process_row(test_part))
    y_train_parts = np.array(y_train_parts)
    train_parts = np.array(train_parts)
    test_parts = np.array(test_parts)

0it [00:00, ?it/s]/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
9988it [01:40, 98.95it/s]

In [152]:
X_train, X_test, y_train, y_test = train_test_split(train_parts, y_train_parts, test_size=0.3)

In [153]:
# fill nan
def fill_nan(data):
    return [np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x) for x in data]

X_train = fill_nan(X_train)
X_test = fill_nan(X_test)

In [156]:
X_train = pad_sequences(X_train)
y_train = pad_sequences(y_train)

y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

X_train.shape, y_train.shape

((7000, 280, 39), (7000, 280, 1))

In [157]:
def create_model():
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    
    x = LSTM(10, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(1))(x)

    model = Model(inputs, outputs)
    
    model.compile('adam', 'mean_absolute_error')
    
    return model

In [ ]:
model = create_model()
model.fit(X_train, y_train, batch_size=64, nb_epoch=10, validation_split=0.1, verbose=1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 6300 samples, validate on 700 samples
Epoch 1/10
6300/6300 [==============================] - 43s - loss: 12.5019 - val_loss: 12.7978
Epoch 2/10
2688/6300 [===========>..................] - ETA: 24s - loss: 12.2531